In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
from scipy.optimize import curve_fit
import tkinter as tk
from tkinter import filedialog

matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
def gaussian(x, A, mu, sig):
    return A*exp(-power(x - mu, 2.) / (2 * power(sig, 2.)))

In [178]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

In [5]:
# sélection des fichiers (Python 3)
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(initialdir = '/home/adrian/Documents/Mesures',multiple=True)
print(file_path)

# ('/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_000.dat',
# '/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_001.dat',
# '/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_002.dat')

# ('/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_000.dat', '/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_001.dat')

('/home/adrien/Documents/Mesures/Neutres/221026/221026_Fluo_neutre_DL423_000.dat', '/home/adrien/Documents/Mesures/Neutres/221026/221026_Fluo_neutre_DL423_001.dat', '/home/adrien/Documents/Mesures/Neutres/221026/221026_Fluo_neutre_DL423_002.dat')


In [174]:
# Array rectangle
# Allocation à l'avance

filename=file_path
m = loadtxt(file_path[0],usecols=(2),delimiter='\t',skiprows=0,unpack=True).size
n = len(file_path)
shape = (m, n)

TIME = zeros(shape)
PM = zeros(shape)
CH2 = zeros(shape)

for k in range(0,len(filename)):
    TIME[:,k], PM[:,k] = loadtxt(file_path[k],usecols=(0,1),delimiter='\t',skiprows=0,unpack=True)

In [175]:
fond = mean(PM[-100:,:]) # amin(PM,axis=0)
# fond = mean(PM[-100:,:],axis=0)
PM = PM - fond
fmax = array([709.08070, 0, 299792458/1.0002818/422.67113e-9 *1e-12]) # 709.08070
fmin = array([709.07780, 0, 299792458/1.0002818/422.67282e-9 *1e-12]) # 709.07780
ftrans = 709.07800
FREQ = (TIME/amax(TIME,axis=0)*(fmax-fmin)+fmin - ftrans) # 
max_FREQ = argmax(PM,axis=0)
print(max_FREQ)

[151 114 101]


In [176]:
# plot for simple scan

to_plot = 2
print(file_path[to_plot])
fig = figure('Fluo_neutres',clear='True')
ax = fig.add_subplot(1, 1, 1)

ax.plot(FREQ[:,to_plot]*1e6,PM[:,to_plot],'+',color='xkcd:prussian blue',label='423 fluo.') # TIME[:,to_plot]

xfit = linspace(0,1500,1000)
pstart = [25000,650,200]
popt, pcov = curve_fit(gaussian,
                       FREQ[max_FREQ[to_plot]-50:max_FREQ[to_plot]+50,to_plot]*1e6,
                       PM[max_FREQ[to_plot]-50:max_FREQ[to_plot]+50,to_plot],p0=pstart)
ax.plot(xfit,gaussian(xfit,*popt))

# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5000))

# xlabel(r't [ms]')
xlabel(r'$\Delta f_{423}$ [MHz]')
ylabel('fluo. [cts/5ms]')
ax.legend(fontsize=18)

grid()
tight_layout()

/home/adrien/Documents/Mesures/Neutres/221026/221026_Fluo_neutre_DL423_002.dat


In [196]:
FWHM = 2*sqrt(2*log(2))*popt[2]*1e-6
Temperature = 300*40.078*(299792458/1.7/2230*FWHM/ftrans)**2
print(FWHM*1e6,Temperature)
# Ce calcul ne prend pas en compte la vitesse moyenne du jet de neutres

260.42912114290027 10.14266779589865
